In [1]:
# Import Requirements
import pandas as pd
from tqdm import tqdm as tq
import numpy as np
import matplotlib.pyplot as plt
from pre_processing_posts import pre_processing_posts as pre_posts
import datetime

Matplotlib is building the font cache; this may take a moment.


[RQ7] What's the probability that a post receives more than 20% "likes" of the number of followers a user has?

In [17]:
#Drop NaN valued rows
df_profiles=pd.read_csv('/home/ec2-user/SageMaker/Data/instagram_profiles.csv',usecols=['sid','followers'],sep='\t')
df_profiles=df_profiles.dropna()

In [19]:
'Arguments':#This function takes as input value of number of like(x) and Number of followers(y) for a user
'Returns': '1 if the number of likes is greater than 20% of the user else 0 the other way round'
def compare_likes_followers(x,y):
    if x>(0.2)*y:
        return 1
    else:
        return 0

In [27]:

for i,chunk in tq(enumerate(pd.read_csv('/home/ec2-user/SageMaker/Data/instagram_posts.csv',sep='\t',usecols=['sid_profile','numbr_likes'],chunksize=100000,keep_default_na=True,na_values=' '))):
        chunk=chunk.dropna(how='any')
        chunk=chunk.rename(columns={'sid_profile':'sid'})
        df_iter=pd.merge(chunk,df_profiles,on='sid')
        try:
            x=df_iter.apply( lambda row: compare_likes_followers(int(row['numbr_likes']),int(row['followers'])),axis=1)
        except:
            print('it still does not drop '' ')
        sum1=sum1+sum(x) if i!=0 else sum(x)
        size=size+len(x) if i!=0 else len(x) 
        
        #print(sum1,size)
        
print('the probability that a post receives more than 20% "likes" of the number of followers a user has is '+str(sum1/size*100)+'%')       
        
        

428it [15:19,  2.15s/it]


In [30]:

% store 

the probability that a post receives more than 20% "likes" of the number of followers a user has is 15.235878494512937%


[RQ7.2] Do users usually return to locations? Extract the probability that a user returns to a site after having posted it in the past. Does that probability make sense to you? Explain why or why not.

Starting idea is to to get use Locations. csv and Posts.csv simultaneously. 
Since Locations dataset is small in size, I read the complete Location dataset in one-go. Whereas for the Posts dataset, I read the dataset in multiple chunks.

*Approach*:
1. While reading the dataset in chunk for Posts I extract the profile id and location_id columns whereas just the 'id' column for the location dataset.

2. Now, for each chunk I try to match the Posts['location_id'] with Location['id'] so that I can query location information from the matched location['id'].

3. Now each user would have a unique 'Profile Id' and through which I can see how many different locations the user has visited and see if the user went back to the sampe place again.

4. For each user, hence I calculate two terms:Freq_places (The places which he has vistited more than once,total_places:no. of unique places he has visited. 

5. Sum the two variable for all the users and calculate the required probability i.e. probability of users that return back to the places that they have visited once.

In [ ]:
#Drop NaN valued rows #Profiles
df_locations=pd.read_csv('/home/ec2-user/SageMaker/Data/instagram_locations.csv',usecols=['id'],sep='\t')
df_locations.info()

In [ ]:

for i,chunk in tq(enumerate(pd.read_csv('/home/ec2-user/SageMaker/Data/instagram_posts.csv',sep='\t',usecols=['profile_id','location_id'],chunksize=100000,keep_default_na=True))):
           df_final=pd.merge(chunk,df_locations,left_on='location_id',right_on='id')
           df_final=df_final.drop(['id'],axis=1)
           x=np.concatenate([x,df_final.values],axis=0) if i!=0 else df_final.values

In [ ]:
profile_id=x.copy()[:,0]
location_id=x.copy()[:,1]

In [ ]:
p_unique_elements,p_unique_indices,p_indices,p_counts=np.unique(profile_id,return_inverse=True,return_index=True,return_counts=True)
l_unique_elements,l_unique_indices,l_indices,l_counts=np.unique(location_id,return_inverse=True,return_index=True,return_counts=True)


In [ ]:
for i,j in enumerate(p_unique_indices):
    _,b=np.unique(location_id[p_indices==j],return_counts=True)
    freq_places=freq_places+sum(b[b>1]) if i!=0 else sum(b[b>1])
    total_places=total_places+len(b) if i!=0 else len(b)
